In [2]:
from common_genai_utils.geminihelper import get_llm, get_retriever
llm = get_llm()
retriever = get_retriever(llm_model=llm)
retriever
results = retriever.invoke("what is langgraph?")
results

model name used -  models/gemini-1.5-pro


[]

In [7]:
from dotenv import load_dotenv

loaded_env = load_dotenv(dotenv_path="../.env", override=True)
print("Loaded env variables from parent directory -> ", loaded_env)

Loaded env variables from parent directory ->  True


In [22]:
import os

from langchain_google_genai.google_vector_store import GoogleVectorStore
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

import nest_asyncio
nest_asyncio.apply()

def get_vector_db_retriever(llm_model):
    persist_path = os.path.join("./temp/", "union.parquet")
    print(" get_vector_db_retriever for model ", llm_model)
    embd = GoogleGenerativeAIEmbeddings(model=llm_model.model)
    # If vector store exists, then load it
    corpus_id="docs-langchain-001"
    if os.path.exists(persist_path):
        print("loading existing vectorstore ")
        vectorstore = GoogleVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet", 
            corpus_id=corpus_id
        )
        return vectorstore.as_retriever(lambda_mult=0)
    print("index LangSmith documents and create new vector store : started")
    # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml")
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=5000, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)
    print("doc_splits = ", type(doc_splits) )
    for doc in doc_splits: 
        print("DOcument = ", type(doc) )
    vectorstore = GoogleVectorStore.from_documents(documents=doc_splits, 
                                    embedding=embd, 
                                    persist_path=persist_path, 
                                    corpus_id=corpus_id)
    #vectorstore = GoogleVectorStore.create_corpus(corpus_id=corpus_id)
    #vectorstore.add_documents(doc_splits)
    #vectorstore.persist()
    print("index LangSmith documents and create new vector store : completed ")
    return vectorstore.as_retriever(lambda_mult=0)

In [23]:
ret = get_vector_db_retriever(llm_model=llm)

 get_vector_db_retriever for model  model='models/gemini-1.5-pro' google_api_key=SecretStr('**********') client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x120d88790> default_metadata=()
index LangSmith documents and create new vector store : started


Fetching pages: 100%|#####################################################################################################################################################################################| 219/219 [00:20<00:00, 10.70it/s]


doc_splits =  <class 'list'>
DOcument =  <class 'langchain_core.documents.base.Document'>
DOcument =  <class 'langchain_core.documents.base.Document'>
DOcument =  <class 'langchain_core.documents.base.Document'>
DOcument =  <class 'langchain_core.documents.base.Document'>
DOcument =  <class 'langchain_core.documents.base.Document'>
DOcument =  <class 'langchain_core.documents.base.Document'>
DOcument =  <class 'langchain_core.documents.base.Document'>
DOcument =  <class 'langchain_core.documents.base.Document'>
DOcument =  <class 'langchain_core.documents.base.Document'>
DOcument =  <class 'langchain_core.documents.base.Document'>
DOcument =  <class 'langchain_core.documents.base.Document'>
DOcument =  <class 'langchain_core.documents.base.Document'>
DOcument =  <class 'langchain_core.documents.base.Document'>
DOcument =  <class 'langchain_core.documents.base.Document'>
DOcument =  <class 'langchain_core.documents.base.Document'>
DOcument =  <class 'langchain_core.documents.base.Docume

NotImplementedError: Must provide an existing corpus ID and document ID